1. Add autograd Fairness gradient in backward to closed-form

2. Verify formula of MAD and Acc Parity on Group

3. Let BETA = ALPHA

4. Also report training time

5. For group, report group-wise performance (MSE and Decision Solution&Objective)

6. For Fold-OPT Change PGD closed-form to solver.

7. Report fairness value when lambda = 0

8.

In [123]:
from src.utils.myOptimization import (
    solveGroupProblem, closed_form_group_alpha, AlphaFairnesstorch,
    solveIndProblem, solve_closed_form, solve_coupled_group_alpha, compute_coupled_group_obj
)
from src.utils.myPrediction import generate_random_features, customPredictionModel
from src.utils.plots import visLearningCurve
from src.fairness.cal_fair_penalty import atkinson_loss, mean_abs_dev, compute_group_accuracy_parity

from src.utils.myOptimization import AlphaFairness, AlphaFairnesstorch, solve_coupled_group_grad, compute_gradient_closed_form
from src.utils.myOptimization import compute_individual_gradient_analytical, compute_group_gradient_analytical
# ------------------------------------------------------------------
import numpy as np
import cvxpy as cp
import torch
import torch.nn as nn
from torch.autograd import Function
import pandas as pd

from src.utils.features import get_all_features
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import warnings

warnings.filterwarnings("ignore")


## Define Alpha & Q

In [124]:
# Save to json
import json
params = {
    "n_sample": 5000 ,
    "alpha": 2,
    "beta": 2.5,
    "Q": 1000,
    "epochs": 100,
    "lambdas": 1.0,
    "lr": 0.01
}

# with open("E:\\User\\Stevens\\MyRepo\\Organized-FDFL\\src\\models\\config_CF.json", "w") as f:
#     json.dump(params, f, indent=4)

# import json

# with open("E:\\User\\Stevens\\MyRepo\\Organized-FDFL\\src\\models\\config_CF.json", "r") as f:
#     params = json.load(f)

n_sample = params["n_sample"]
alpha    = params["alpha"]
beta     = params["beta"]
Q        = params["n_sample"]//2
epochs   = params["epochs"]
lambdas  = params["lambdas"]
lr       = params["lr"]
print(Q)


2500


In [125]:
df = pd.read_csv('/Users/dennis/Downloads/2024-fall/research/Fairness-Decision-Focused-Loss/Organized-FDFL/src/data/data.csv')
df = df.sample(n=n_sample,random_state=42)

# Normalized cost to 0.1-10 range
cost = np.array(df['cost_t_capped'].values) * 10
cost = np.maximum(cost, 0.1)

# All features, standardized
features = df[get_all_features(df)].values
scaler = StandardScaler()
features = scaler.fit_transform(features)

# True benefit, predictor label normalzied to 1-100 range
benefit = np.array(df['benefit'].values) * 100
benefit = np.maximum(benefit, 0.1) 

# Group labels, 0 is White (Majority), 1 is Black
race = np.array(df['race'].values)

gainF = np.ones_like(benefit)

## Prediction Model

In [126]:
class FairRiskPredictor(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.1):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1),
            # nn.ReLU(),
            # nn.Dropout(dropout_rate),
            # nn.Linear(64, 1),
            nn.Softplus()
        )
            
    def forward(self, x):
        return self.model(x).squeeze(-1)

## JVP calculation (test)

In [127]:
def solve_coupled_group_jvp(b, c, group_idx, Q, alpha, beta, v):
    """
    Computes the vector-Jacobian product v @ J for the coupled group-alpha problem
    without explicitly forming the full Jacobian matrix J.
    Complexity: O(n) for each element of the output, avoiding O(n^2).
    """
    # Ensure inputs are NumPy arrays
    b, c, group_idx, v = map(np.asarray, [b, c, group_idx, v])
    n = len(b)
    final_grad = np.zeros(n)

    # --- 1. Forward Pass: Pre-compute terms from the solver ---
    # This part is identical to the start of the original _grad function
    if beta > 1:
        gamma = beta - 2 + alpha - alpha * beta
        psi_s_exp_factor = (2 - alpha) / gamma
    else: # beta < 1
        gamma = beta + alpha - alpha * beta
        psi_s_exp_factor = -alpha / gamma

    d_star = solve_coupled_group_alpha(b, c, group_idx, Q, alpha, beta)
    unique_groups = np.unique(group_idx)
    S, H, Psi = {}, {}, {}
    for k in unique_groups:
        mask = (group_idx == k)
        G_k, b_k, c_k = np.sum(mask), b[mask], c[mask]
        S[k] = np.sum((c_k**(-(1-beta)/beta)) * (b_k**((1-beta)/beta)))
        H[k] = np.sum((c_k**((beta-1)/beta)) * (b_k**((1-beta)/beta)))
        const_factor = (beta - 1) if beta > 1 else (1 - beta)
        if beta > 1:
            Psi[k] = (S[k]**psi_s_exp_factor) * (const_factor**((alpha-2)/gamma))
        else:
            Psi[k] = (G_k**((alpha-1)/gamma)) * (S[k]**psi_s_exp_factor) * (const_factor**(alpha/gamma))
    Xi = np.sum([H[k] * Psi[k] for k in unique_groups])
    phi_all = (c**(-1/beta)) * (b**((1-beta)/beta))

    # --- 2. Compute the scalar term `Σᵢ vᵢ * dᵢ*` ---
    v_dot_d_star = np.dot(v, d_star)

    # --- 3. Backward Pass: Loop through each prediction `b_j` to get the j-th grad component ---
    for j in range(n):
        m = group_idx[j] # Group of the variable b_j

        # --- Calculate `∂Ξ/∂bⱼ` (same as before) ---
        dS_m_db_j = ((1-beta)/beta) * (c[j]**(-(1-beta)/beta)) * (b[j]**((1-2*beta)/beta))
        dH_m_db_j = ((1-beta)/beta) * (c[j]**((beta-1)/beta)) * (b[j]**((1-2*beta)/beta))
        dPsi_m_db_j = (psi_s_exp_factor / S[m]) * Psi[m] * dS_m_db_j
        dXi_db_j = dH_m_db_j * Psi[m] + H[m] * dPsi_m_db_j

        # --- Calculate the JVP-specific term `Σᵢ vᵢ * (∂Nᵢ/∂bⱼ)` ---
        # ∂Nᵢ/∂bⱼ = Q * ( (∂Ψₖ/∂bⱼ) * φᵢ + Ψₖ * (∂φᵢ/∂bⱼ) )
        # We need to sum vᵢ * (∂Nᵢ/∂bⱼ) over all i
        sum_v_dN_db_j = 0
        dphi_j_db_j = ((1-beta)/beta) * (c[j]**(-1/beta)) * (b[j]**((1-2*beta)/beta))

        # The derivative ∂Ψₖ/∂bⱼ is only non-zero if k == m
        # The derivative ∂φᵢ/∂bⱼ is only non-zero if i == j
        # This makes the sum sparse and efficient to compute
        sum_v_dN_db_j += Q * dPsi_m_db_j * np.dot(v[group_idx == m], phi_all[group_idx == m])
        sum_v_dN_db_j += Q * Psi[m] * v[j] * dphi_j_db_j

        # --- 4. Assemble the final gradient component ---
        final_grad[j] = (1/Xi) * sum_v_dN_db_j - (dXi_db_j / Xi) * v_dot_d_star

    return final_grad

## Dataloader

In [128]:
from torch.utils.data import Dataset

def to_numpy_1d(x):
    """Return a 1-D NumPy array; error if the length is not > 1."""
    if isinstance(x, torch.Tensor):
        x = x.detach().cpu().numpy()
    x = np.asarray(x).reshape(-1)
    assert x.ndim == 1, f"expected 1-D, got shape {x.shape}"
    return x

class optDataset(Dataset):
    def __init__(self, feats, risk, gainF, cost, race, alpha=alpha, Q=Q):
        # Store as numpy arrays for now
        self.feats = feats
        self.risk = risk
        self.gainF = gainF
        self.cost = cost
        self.race = race


        # Call optmodel (expects numpy arrays)
        sol_group = solve_coupled_group_alpha(self.risk, self.cost, self.race, Q=Q, alpha=alpha, beta=beta)
        obj_group = compute_coupled_group_obj(sol_group, self.risk, self.race, alpha=alpha, beta=beta)

        sol_ind, _ = solve_closed_form(self.gainF, self.risk, self.cost, alpha=alpha, Q=Q)

        obj_ind = AlphaFairness(self.risk*sol_ind,alpha=alpha)

        # Convert everything to torch tensors for storage
        self.feats = torch.from_numpy(self.feats).float()
        self.risk = torch.from_numpy(self.risk).float()
        self.gainF = torch.from_numpy(self.gainF).float()
        self.cost = torch.from_numpy(self.cost).float()
        self.race = torch.from_numpy(self.race).float()
        self.sol_ind = torch.from_numpy(sol_ind).float()
        self.sol_group = torch.from_numpy(sol_group).float()

        # to array
        obj_group = np.array(obj_group)
        self.obj_group = torch.from_numpy(obj_group).float()
        self.obj_ind = torch.tensor(obj_ind).float()

    def __len__(self):
        return len(self.feats)

    # def __getitem__(self, idx):
    #     return self.feats, self.risk, self.gainF, self.cost, self.race, self.sol, self.obj

    def __getitem__(self, idx):
        return (
            self.feats[idx],
            self.risk[idx],
            self.gainF[idx],
            self.cost[idx],
            self.race[idx],
            self.sol_ind[idx],
            self.sol_group[idx],
            self.obj_group,
            self.obj_ind
        )


In [129]:
optmodel_group = solve_coupled_group_alpha
optmodel_ind = solve_closed_form

# Perform train-test split
feats_train, feats_test, gainF_train, gainF_test, b_train, b_test, cost_train, cost_test, race_train, race_test = train_test_split(
    features, gainF, benefit, cost, df['race'].values, test_size=0.5, random_state=2
)

print(f"Train size: {feats_train.shape[0]}")
print(f"Test size: {feats_test.shape[0]}")

dataset_train = optDataset(feats_train, b_train, gainF_train, cost_train, race_train, alpha=alpha, Q=Q)
dataset_test = optDataset(feats_test, b_test, gainF_test, cost_test, race_test, alpha=alpha, Q=Q)


dataloader_train = DataLoader(dataset_train, batch_size=len(dataset_train), shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=len(dataset_train), shuffle=False)

predmodel = FairRiskPredictor(feats_train.shape[1])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
predmodel.to(device)


# Get a batch from the dataloader
for batch in dataloader_train:
    names = [
        "feats", "risk", "gainF", "cost", "race",
        "sol_ind", "sol_group", "obj_group", "obj_ind"
    ]
    for name, item in zip(names, batch):
        # Only show first five elements for feats
        if name == "feats":
            print(f"First five {name}: {item[:1, :5]}")
        else:
            print(f"{name}: {item[:1]}")
    break

Train size: 2500
Test size: 2500
First five feats: tensor([[-1.3127, -0.1998, -0.3537, -0.4862,  1.7943]])
risk: tensor([0.1000])
gainF: tensor([1.])
cost: tensor([0.1000])
race: tensor([0.])
sol_ind: tensor([12.3363])
sol_group: tensor([11.4406])
obj_group: tensor([-1190.4932])
obj_ind: tensor([-1642.7346])


## Regret Loss nn.Module Gemini Version



In [ ]:
def _calculate_loss_and_decision(pred_r, true_r, gainF, cost, race, Q, alpha, beta, lambdas, fairness_type, group, **kwargs):
    """
    Helper function to compute loss. Detaches inputs to prevent this logic from being part of the graph,
    as its gradient is handled manually in the backward pass.
    """
    # Use detached tensors for calculation
    pred_r_d, true_r_d, gainF_d, cost_d, race_d = map(
        lambda t: t.detach(), [pred_r, true_r, gainF, cost, race]
    )
    pred_r_np, true_r_np, gainF_np, cost_np, race_np = map(to_numpy_1d, [pred_r_d, true_r_d, gainF_d, cost_d, race_d])

    try:
        if group:
            d_hat_np = solve_coupled_group_alpha(pred_r_np, cost_np, race_np, Q, alpha, beta)
            d_star_np = solve_coupled_group_alpha(true_r_np, cost_np, race_np, Q, alpha, beta)
            obj_val_at_d_hat = compute_coupled_group_obj(d_hat_np, true_r_np, race_np, alpha, beta)
            obj_val_at_d_star = compute_coupled_group_obj(d_star_np, true_r_np, race_np, alpha, beta)
        else:
            d_hat_np, _ = solve_closed_form(gainF_np, pred_r_np, cost_np, alpha, Q)
            d_star_np, _ = solve_closed_form(gainF_np, true_r_np, cost_np, alpha, Q)
            obj_val_at_d_hat = AlphaFairness(true_r_np * d_hat_np, alpha)
            obj_val_at_d_star = AlphaFairness(true_r_np * d_star_np, alpha)

        # Ensure regret is not negative due to solver noise
        regret_loss = torch.tensor(max(0, obj_val_at_d_star - obj_val_at_d_hat), dtype=pred_r.dtype, device=pred_r.device)

    except (ValueError, cp.error.SolverError, np.linalg.LinAlgError) as e:
        print(f"Warning: Solver failed: {e}")
        return torch.tensor(0.0), torch.tensor(0.0), None

    # Use the original tensors (with graph) for fairness calculation for autograd
    fairness_penalty = torch.tensor(0.0, device=pred_r.device)
    if fairness_type != 'none':
        mode = 'between' if group else 'individual'
        if fairness_type == 'atkinson': fairness_penalty = atkinson_loss(pred_r, true_r, race=race, beta=beta, mode=mode)
        elif fairness_type == 'mad': fairness_penalty = mean_abs_dev(pred_r, true_r, race=race, mode=mode)
        elif fairness_type == 'acc_parity' and group: fairness_penalty = compute_group_accuracy_parity(pred_r, true_r, race)

    total_loss = regret_loss + lambdas * fairness_penalty
    return total_loss, fairness_penalty, d_hat_np

In [ ]:
class RegretFairnessLoss(Function):
    @staticmethod
    def forward(ctx, pred_r, true_r, gainF, cost, race, Q, alpha, beta, lambdas, fairness_type, group, grad_method):
        # We need the graph for the fairness penalty part for autograd
        _, fairness_penalty_for_grad, d_hat_np = _calculate_loss_and_decision(
            pred_r, true_r, gainF, cost, race, Q, alpha, beta, lambdas, fairness_type, group
        )
        # But the regret part of the loss should be detached
        regret_loss, _, _ = _calculate_loss_and_decision(
            pred_r.detach(), true_r, gainF, cost, race, Q, alpha, beta, 0, 'none', group
        )
        # The final loss combines the detached regret with the graph-connected fairness penalty
        total_loss = regret_loss + lambdas * fairness_penalty_for_grad

        d_hat = torch.from_numpy(d_hat_np).to(pred_r.device, dtype=pred_r.dtype) if d_hat_np is not None else None
        ctx.save_for_backward(pred_r, true_r, gainF, cost, race, d_hat)
        ctx.params = {'Q': Q, 'alpha': alpha, 'beta': beta, 'lambdas': lambdas, 'fairness_type': fairness_type, 'group': group, 'grad_method': grad_method}
        return total_loss

    @staticmethod
    def backward(ctx, grad_output):
        pred_r, true_r, gainF, cost, race, d_hat = ctx.saved_tensors
        params = ctx.params
        if d_hat is None:
            return (torch.zeros_like(pred_r),) + (None,) * 11

        # PyTorch autograd will automatically handle the gradient for the fairness term.
        # We only need to compute and return the gradient for the regret term.
        grad_regret = torch.zeros_like(pred_r)

        if params['grad_method'] == 'closed-form':
            try:
                if params['group']:
                    # --- EFFICIENT JVP PATH ---
                    pred_r_np, cost_np, race_np = map(to_numpy_1d, [pred_r, cost, race])
                    grad_obj_wrt_d_hat = compute_group_gradient_analytical(d_hat, true_r, race, params['alpha'], params['beta'])
                    grad_obj_wrt_d_hat_np = to_numpy_1d(grad_obj_wrt_d_hat)
                    v_dot_J = solve_coupled_group_jvp(pred_r_np, cost_np, race_np, params['Q'], params['alpha'], params['beta'], v=grad_obj_wrt_d_hat_np)
                    grad_regret = -torch.from_numpy(v_dot_J).to(pred_r.device, dtype=pred_r.dtype)
                else:
                    # --- Individual case (already efficient) ---
                    pred_r_np, cost_np, gainF_np = map(to_numpy_1d, [pred_r, cost, gainF])
                    jac = compute_gradient_closed_form(gainF_np, pred_r_np, cost_np, params['alpha'], params['Q'])
                    grad_obj_wrt_d_hat = (true_r * gainF) ** (1 - alpha) * pred_r ** (-alpha)
                    jac_tensor = torch.from_numpy(jac).to(pred_r.device, dtype=pred_r.dtype)
                    grad_obj_tensor = grad_obj_wrt_d_hat.to(dtype=pred_r.dtype, device=pred_r.device)
                    grad_regret = -grad_obj_tensor @ jac_tensor

            except (ValueError, TypeError, np.linalg.LinAlgError) as e:
                print(f"Warning: Closed-form gradient failed: {e}. Returning zero grad for regret.")

        elif params['grad_method'] == 'finite-diff':
            epsilon = 1e-5
            for i in range(pred_r.numel()):
                p_plus, p_minus = pred_r.detach().clone(), pred_r.detach().clone()
                p_plus[i] += epsilon; p_minus[i] -= epsilon
                loss_plus, _, _ = _calculate_loss_and_decision(p_plus, true_r, gainF, cost, race, **params)
                loss_minus, _, _ = _calculate_loss_and_decision(p_minus, true_r, gainF, cost, race, **params)
                grad_regret[i] = (loss_plus - loss_minus) / (2 * epsilon)
            # This numerically approximates the gradient of the *entire* loss.
            # We return it directly, as autograd won't have a better value.
            return (grad_output * grad_regret, None, None, None, None, None, None, None, None, None, None, None)

        # The final gradient passed to the optimizer is the sum of the autograd part (fairness)
        # and our manually computed part (regret). PyTorch handles this addition automatically.
        return (grad_output * grad_regret, None, None, None, None, None, None, None, None, None, None, None)


# ==============================================================================
# ===== 3. Final nn.Module Wrapper
# ==============================================================================

class FDFLLoss(nn.Module):
    def __init__(self, Q, alpha, beta, lambdas, fairness_type, group, grad_method='closed-form'):
        super().__init__()
        self.Q, self.alpha, self.beta, self.lambdas = Q, alpha, beta, lambdas
        self.fairness_type, self.group, self.grad_method = fairness_type, group, grad_method

    def forward(self, pred_r, true_r, gainF, cost, race):
        return RegretFairnessLoss.apply(pred_r, true_r, gainF, cost, race, self.Q, self.alpha, self.beta, self.lambdas, self.fairness_type, self.group, self.grad_method)


# Training Gemini Version

In [132]:
def train_model_regret(
        X_train, y_train, race_train, cost_train, gainF_train,
        X_test,  y_test,  race_test,  cost_test, gainF_test,
        model_class, input_dim,
        alpha, beta, Q,
        lambda_fair=0.0, fairness_type="none", group=True, grad_method='closed-form',
        num_epochs=30, lr=1e-2, batch_size=None,
        dropout_rate=0.1, weight_decay=1e-4,
        device=torch.device("cpu")):
    """
    Train a predictor using the custom RegretFairnessLoss.
    """
    # -------------------------- Tensors and Device ---------------------
    tensors = [X_train, y_train, race_train, cost_train, gainF_train,
               X_test, y_test, race_test, cost_test, gainF_test]
    X_train, y_train, race_train, cost_train, gainF_train, \
    X_test, y_test, race_test, cost_test, gainF_test = [
        torch.tensor(t, dtype=torch.float32, device=device) if not isinstance(t, torch.Tensor) else t.to(device)
        for t in tensors
    ]

    # -------------------------- Dataloaders ------------------------
    train_ds = TensorDataset(X_train, y_train, race_train, cost_train, gainF_train)
    if batch_size is None: batch_size = len(train_ds)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

    # -------------------------- Model, Optimizer, and Loss ---------------
    model = model_class(input_dim, dropout_rate=dropout_rate).to(device)
    optim = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    crit = FDFLLoss(Q, alpha, beta, lambda_fair, fairness_type, group, grad_method)

    # -------------------------- Logs -------------------------------
    loss_log, mse_log, regret_log, fairness_log = [], [], [], []

    # -------------------------- Training Loop ----------------------
    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_loss = 0.0

        for x_b, y_b, r_b, c_b, g_b in train_loader:
            pred_b = model(x_b).squeeze()
            pred_b = torch.clamp(pred_b, min=1e-4) # Ensure predictions are positive

            loss = crit(pred_b, y_b, g_b, c_b, r_b)

            optim.zero_grad()
            if loss.requires_grad:
                loss.backward()
                optim.step()
            epoch_loss += loss.item() * x_b.size(0)

        loss_log.append(epoch_loss / len(train_ds))

        # ----------------- Evaluation on the held-out set ----------
        model.eval()
        with torch.no_grad():
            pred_test = model(X_test).squeeze().clamp(min=1e-4)
            mse_val = ((pred_test - y_test).pow(2)).mean().item()
            mse_log.append(mse_val)

            # --- Calculate Regret on Test Set ---
            _, _, d_pred_np = _calculate_loss_and_decision(
                pred_test, y_test, gainF_test, cost_test, race_test,
                Q, alpha, beta, 0, 'none', group
            )
            _, _, d_true_np = _calculate_loss_and_decision(
                y_test, y_test, gainF_test, cost_test, race_test,
                Q, alpha, beta, 0, 'none', group
            )
            
            if d_pred_np is not None and d_true_np is not None:
                if group:
                    true_obj = compute_coupled_group_obj(d_true_np, to_numpy_1d(y_test), to_numpy_1d(race_test), alpha, beta)
                    pred_obj = compute_coupled_group_obj(d_pred_np, to_numpy_1d(y_test), to_numpy_1d(race_test), alpha, beta)
                else:
                    true_obj = AlphaFairness(to_numpy_1d(y_test) * d_true_np, alpha)
                    pred_obj = AlphaFairness(to_numpy_1d(y_test) * d_pred_np, alpha)
                norm_regret = (true_obj - pred_obj) / (abs(true_obj) + 1e-7)
            else:
                norm_regret = np.nan # Solver failed
            regret_log.append(norm_regret)


            # --- Calculate Fairness on Test Set ---
            fair_val = 0.0
            if fairness_type != 'none':
                mode = 'between' if group else 'individual'
                if fairness_type == "acc_parity" and group: fair_val = compute_group_accuracy_parity(pred_test, y_test, race_test).item()
                elif fairness_type == "atkinson": fair_val = atkinson_loss(pred_test, y_test, race_test, beta=beta, mode=mode).item()
                elif fairness_type == "mad": fair_val = mean_abs_dev(pred_test, y_test, race_test, mode=mode).item()
            fairness_log.append(fair_val)

        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch {epoch:02d}/{num_epochs} | Train-Loss {loss_log[-1]:.4f} | Test-MSE {mse_val:.4f} | Regret {norm_regret:.4f} | Fair {fair_val:.4f}")

    return model, {"loss_log": loss_log, "mse_log": mse_log, "regret_log": regret_log, "fairness_log": fairness_log}



In [163]:
hyperparams = {
    "alpha": 0.5,
    "beta": 0.5,
    "Q": Q,
    "lambda_fair": 0,
    "fairness_type": "atkinson",   
    "group": False,            # Set to True for group fairness, False for individual
    "grad_method": "closed-form",
    "num_epochs": 50,        
    "lr": 0.001,
    "batch_size": len(b_train),
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

final_model, logs = train_model_regret(
    X_train=feats_train, y_train=b_train, race_train=race_train, cost_train=cost_train, gainF_train=gainF_train,
    X_test=feats_test, y_test=b_test, race_test=race_test, cost_test=cost_test, gainF_test=gainF_test,
    model_class=FairRiskPredictor,
    input_dim=feats_train.shape[1],
    **hyperparams
)



Epoch 01/50 | Train-Loss 7332.9580 | Test-MSE 346.3866 | Regret 0.3142 | Fair 0.6579
Epoch 10/50 | Train-Loss 6683.3926 | Test-MSE 343.6100 | Regret 0.2863 | Fair 0.6592
Epoch 20/50 | Train-Loss 6130.1113 | Test-MSE 340.2187 | Regret 0.2639 | Fair 0.6603
Epoch 30/50 | Train-Loss 5745.0234 | Test-MSE 337.0707 | Regret 0.2497 | Fair 0.6608
Epoch 40/50 | Train-Loss 5487.3164 | Test-MSE 334.3893 | Regret 0.2410 | Fair 0.6610
Epoch 50/50 | Train-Loss 5314.2559 | Test-MSE 332.1207 | Regret 0.2352 | Fair 0.6610


In [144]:
hyperparams = {
    "alpha": alpha,
    "beta": beta,
    "Q": Q,
    "lambda_fair": 1,
    "fairness_type": "atkinson",   
    "group": True,            # Set to True for group fairness, False for individual
    "grad_method": "closed-form",
    "num_epochs": 50,        
    "lr": 0.001,
    "batch_size": len(b_train),
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

final_model, logs = train_model_regret(
    X_train=feats_train, y_train=b_train, race_train=race_train, cost_train=cost_train, gainF_train=gainF_train,
    X_test=feats_test, y_test=b_test, race_test=race_test, cost_test=cost_test, gainF_test=gainF_test,
    model_class=FairRiskPredictor,
    input_dim=feats_train.shape[1],
    **hyperparams
)


Epoch 01/50 | Train-Loss 6295.5972 | Test-MSE 346.1301 | Regret 5.3562 | Fair 0.0204
Epoch 10/50 | Train-Loss 5209.8638 | Test-MSE 343.1317 | Regret 4.4941 | Fair 0.0201
Epoch 20/50 | Train-Loss 4566.1587 | Test-MSE 339.7930 | Regret 3.9832 | Fair 0.0198
Epoch 30/50 | Train-Loss 4180.6831 | Test-MSE 337.0753 | Regret 3.6535 | Fair 0.0194
Epoch 40/50 | Train-Loss 3897.2947 | Test-MSE 334.9794 | Regret 3.4328 | Fair 0.0191
Epoch 50/50 | Train-Loss 3672.1292 | Test-MSE 333.2602 | Regret 3.2805 | Fair 0.0187


In [ ]:
# ==============================================================================
# ===== REGRET-BASED TRAINING AND EXPERIMENT HARNESS
# ==============================================================================

def train_many_trials_regret(
        n_trials=10, base_seed=2025, **kwargs):
    """
    Run `train_model_regret` for `n_trials` and average the results.
    Accepts all arguments for `train_model_regret` via **kwargs.
    """
    trials_logs = []
    for t in range(n_trials):
        seed = base_seed + t
        torch.manual_seed(seed)
        np.random.seed(seed)
        _, logs = train_model_regret(**kwargs)
        trials_logs.append(logs)

    keys = ["loss_log", "mse_log", "regret_log", "fairness_log"]
    avg_logs = {}
    for k in keys:
        if trials_logs[0][k] is None or not trials_logs[0][k]:
            avg_logs[k] = [0.0]
            continue
        stack = np.vstack([trial[k] for trial in trials_logs])
        avg_logs[k] = stack.mean(axis=0).tolist()
        std_k = stack.std(axis=0)[-1]
        mean_k = avg_logs[k][-1]
        print(f"[{k.replace('_log','').upper():>8s}] final-epoch mean = {mean_k:.4f}  |  std = {std_k:.4f}")

    return avg_logs


# --- Hyperparameter Grid Definition ---
alphas = [0.5, 1, 1.5, 2]
betas = [0.5, 1, 1.5, 2]
fairness_lambdas = [0, 1.0]
group_settings = [True, False]
grad_methods = ['closed-form', 'finite-diff'] # New parameter
results_list = []

# --- Grid Search Execution ---
for group in group_settings:
    # UPDATED: Added 'acc_parity' for the group case
    if group:
        fairness_types = ['mad', 'acc_parity', 'atkinson']
    else:
        fairness_types = ['mad', 'atkinson']
    
    for grad_method in grad_methods:
        for lam in fairness_lambdas:
            for fairness in fairness_types:
                if lam == 0 and fairness != fairness_types[0]: continue

                for alpha in alphas:
                    current_betas = betas if group else [betas[0]]
                    
                    for beta in current_betas:
                        if group and abs(beta - 1.0) < 1e-9: continue

                        run_params = {
                            'Group': group,
                            'Grad Method': grad_method, # New column
                            'Alpha': alpha,
                            'Beta': beta if group else 'N/A',
                            'Lambda': lam,
                            'Fairness': fairness
                        }
                        print("\n" + "-"*70)
                        print(f"RUNNING EXPERIMENT: {run_params}")
                        print("-"*70)

                        train_args = {
                            'X_train': feats_train, 'y_train': b_train, 'race_train': race_train, 'cost_train': cost_train, 'gainF_train': gainF_train,
                            'X_test': feats_test, 'y_test': b_test, 'race_test': race_test, 'cost_test': cost_test, 'gainF_test': gainF_test,
                            'model_class': FairRiskPredictor, 'input_dim': feats_train.shape[1],
                            'alpha': alpha, 'beta': beta, 'Q': Q,
                            'lambda_fair': lam, 'fairness_type': fairness,
                            'group': group,
                            'grad_method': grad_method, # Pass the current grad method
                            'num_epochs': 30, 'lr': 0.01, 'batch_size': None,
                        }

                        avg_logs = train_many_trials_regret(n_trials=3, **train_args) # Reduced trials for speed

                        final_metrics = run_params.copy()
                        final_metrics['Final Regret'] = avg_logs['regret_log'][-1]
                        final_metrics['Final MSE'] = avg_logs['mse_log'][-1]
                        final_metrics['Final Fairness'] = avg_logs['fairness_log'][-1] if avg_logs.get('fairness_log') else 0.0
                        results_list.append(final_metrics)

# --- Results Presentation ---
results_df = pd.DataFrame(results_list)
# UPDATED: Added 'Grad Method' to the column order
column_order = ['Group', 'Grad Method', 'Alpha', 'Beta', 'Lambda', 'Fairness', 'Final Regret', 'Final MSE', 'Final Fairness']
results_df = results_df[column_order]
latex_table = results_df.to_latex(index=False, caption="Averaged Experimental Results Across Different Parameters.", label="tab:avg_exp_results_expanded", float_format="%.4f")

print("\n\n" + "="*80)
print("                           GRID SEARCH COMPLETE")
print("="*80)
print("\n--- Averaged Results Summary (Pandas DataFrame) ---")
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1200):
    print(results_df)

print("\n\n" + "="*80)
print("--- LaTeX Table Output ---")
print(latex_table)


----------------------------------------------------------------------
RUNNING EXPERIMENT: {'Group': True, 'Grad Method': 'closed-form', 'Alpha': 0.5, 'Beta': 0.5, 'Lambda': 0, 'Fairness': 'None'}
----------------------------------------------------------------------
Epoch 01/30 | Train-Loss 2.1432 | Test-MSE 340.7641 | Regret 0.1190 | Fair 0.0000
Epoch 10/30 | Train-Loss 1.1952 | Test-MSE 316.7935 | Regret 0.0865 | Fair 0.0000
Epoch 20/30 | Train-Loss 0.9423 | Test-MSE 311.2867 | Regret 0.0732 | Fair 0.0000
Epoch 30/30 | Train-Loss 0.7583 | Test-MSE 307.6521 | Regret 0.0642 | Fair 0.0000
Epoch 01/30 | Train-Loss 2.1837 | Test-MSE 341.7054 | Regret 0.1190 | Fair 0.0000
Epoch 10/30 | Train-Loss 1.2376 | Test-MSE 318.7535 | Regret 0.0915 | Fair 0.0000
Epoch 20/30 | Train-Loss 0.9810 | Test-MSE 313.7587 | Regret 0.0756 | Fair 0.0000
Epoch 30/30 | Train-Loss 0.7959 | Test-MSE 308.9955 | Regret 0.0655 | Fair 0.0000
Epoch 01/30 | Train-Loss 2.0675 | Test-MSE 340.0304 | Regret 0.1173 | Fair 

In [152]:
results_df.to_csv("results.csv", index=False)


In [153]:
results_df

,Group,Grad Method,Alpha,Beta,Lambda,Fairness,Final Regret,Final MSE,Final Fairness
0,True,closed-form,0.5,0.5,0.0,None,0.064265,308.300659,0.000000
1,True,closed-form,0.5,1.5,0.0,None,0.053591,290.035441,0.000000
2,True,closed-form,1.0,0.5,0.0,None,0.055729,308.070760,0.000000
3,True,closed-form,1.0,1.5,0.0,None,0.015016,290.205475,0.000000
4,True,closed-form,1.5,0.5,0.0,None,0.076455,308.012095,0.000000
...,...,...,...,...,...,...,...,...,...
83,False,finite-diff,2.0,N/A,1.0,mad,0.726426,299.421651,420.257935
84,False,finite-diff,0.5,N/A,1.0,atkinson,0.185934,310.066538,0.666265
85,False,finite-diff,1.0,N/A,1.0,atkinson,-1.015138,330.118174,0.658891
86,False,finite-diff,1.5,N/A,1.0,atkinson,0.133037,306.971313,0.663845
